<h2>Lab 4 : Part1 ,Classification Regression:</h2>

<hr style="border:2px solid blue;">

<h3>Realised by:</h3>
<ul>
    <li><strong style="color:black;">ELMRABET HANAE</strong></li>
</ul>

<h3>Guided by:</h3>
   <p><strong style="color:blue;">Pr . ELAACHAk LOTFY</strong></p>




<h1>Part 1: Data Collection and Dataset Preparation</h1>

<h2> 1 .Web Scraping with Scrapy/BeautifulSoup</h2>
<ul>Collect text data from several Arabic websites on a specific topic.</ul>
<ul>Prepare the dataset with texts and their corresponding relevance scores.</ul>

In [3]:
import requests
from bs4 import BeautifulSoup
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from nltk.stem import WordNetLemmatizer
import string
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error
import numpy as np

In [4]:

urls = [
    'https://alyaoum24.com/1894091.html',
    'https://alyaoum24.com/1894091.html'
]

def scrape_url(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    paragraphs = soup.find_all('p')
    text = ' '.join([p.get_text(strip=True) for p in paragraphs])
    return text
texts = [scrape_url(url) for url in urls]
scored_texts = [(texts[0], 6), (texts[1], 7.5)]
for i, (text, score) in enumerate(scored_texts):
    print(f'Text {i+1} (Score: {score}):\n{text}\n')

Text 1 (Score: 6):
أفادت مديرية الدراسات والتوقعات المالية التابعة لوزارة الاقتصاد والمالية بأن إنتاج الطاقة الكهربائية على الصعيد الوطني ارتفع بنسبة 5,7 في المائة عند متم الفصل الأول من سنة 2024، مقابل انخفاض نسبته 2,6 في المائة قبل سنة. وأشارت المديرية، في مذكرتها الأخيرة حول الظرفية، إلى أن هذا الأداء يعزى إلى التطور الإيجابي للإنتاج الخاص (زائد 8 في المائة)، وإنتاج الطاقات المتجددة برسم القانون 09 – 13 (+42,8 في المائة)، على الرغم من انكماش إنتاج المكتب الوطني للكهرباء والماء بنسبة 9,9 في المائة. وبالنسبة لمبادلات قطاع الطاقة الكهربائية مع الخارج، فقد انكمش حجم وارداته بنسبة 5,4 في المائة عند متم سنة 2024، بعد ارتفاع بلغت نسبته 211,7 في المائة قبل سنة. وبخصوص حجم الصادرات، فقد ارتفع بنسبة 74,5 في المائة (بعد تسجيل 64,2- في المائة)، وذلك في سياق تزايد حجم الطاقة المسماة الصافية بنسبة 4,2 في المائة (بعد زائد 3,5 في المائة). وفي ما يتعلق باستهلاك الطاقة الكهربائية، فقد ارتفع بنسبة 2,7 في المائة برسم الفصل الأول من 2024، بعد ارتفاع بنسبة 5,4 في المائة قبل سنة. ويعزى هذا النمو في الاسته

<h1>Part 2: Preprocessing NLP Pipeline</h1>

<ul>1. Tokenization</ul>
Splitting the text into individual words or tokens.
<ul>2. Stemming</ul>
Reducing words to their root forms.
<ul>3. Lemmatization</ul>
Reducing words to their base or dictionary form.
<ul>4. Stop Words Removal</ul>
Removing common words that do not contribute much meaning.
<ul>5. Discretization</ul>
Converting continuous data into discrete buckets if needed.

In [5]:

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

stemmer = SnowballStemmer('arabic')
lemmatizer = WordNetLemmatizer()


stop_words = set(stopwords.words('arabic'))

def preprocess_text(text):

    tokens = word_tokenize(text)
    tokens = [word.lower() for word in tokens]
    tokens = [word for word in tokens if word.isalpha()]
    tokens = [word for word in tokens if word not in stop_words]
    stemmed_tokens = [stemmer.stem(word) for word in tokens]
    lemmatized_tokens = [lemmatizer.lemmatize(word) for word in stemmed_tokens]
    return ' '.join(lemmatized_tokens)
preprocessed_texts = [preprocess_text(text) for text in texts]
for i, text in enumerate(preprocessed_texts):
    print(f'Preprocessed Text {i+1}:\n{text}\n')


Preprocessed Text 1:
افاد مدير دراس توقع مال تابع وزار اقتصاد مال بان انتاج طاق كهرباء صعيد وطن ارتفع نسب ماء متم فصل اول سن مقابل انخفاض نسب ماء سن اشار مذكر اخير حول اداء يعزى تطور ايجاب انتاج خاص زايد ماء انتاج طاق متجدد رسم قان ماء رغم انكماش انتاج مكتب وطن كهرباء ماء نسب ماء نسب مبادل قطاع طاق كهرباء فقد انكمش حجم وارد نسب ماء متم سن ارتفاع بلغ نسب ماء سن خصوص حجم فقد ارتفع نسب ماء تسجيل ماء وذل سياق تزايد حجم طاق مسما صاف نسب ماء زايد ماء وف يتعلق استهل طاق فقد ارتفع نسب ماء رسم فصل اول ارتفاع نسب ماء سن يعزى نمو استهلاك ارتفاع مبيع طاق جهد منخفض نسب وذا جهد متوسط نسب وتل موجه موزع نسب ماء votre adresse ne serum pa champ obligatoires sont indiqués avec nom enregistrer mon nom mon et mon site dans le navigateur pour mon prochain commentaire commentaire تحميل تطبيق يوم مجا

Preprocessed Text 2:
افاد مدير دراس توقع مال تابع وزار اقتصاد مال بان انتاج طاق كهرباء صعيد وطن ارتفع نسب ماء متم فصل اول سن مقابل انخفاض نسب ماء سن اشار مذكر اخير حول اداء يعزى تطور ايجاب انتاج خاص زايد ماء انت

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


<h1>Part 3: Model Training</h1>

<h2>1.Train Models Using Different Architectures</h2>
<ul>Recurrent Neural Networks (RNN)</ul>
<ul>Bidirectional RNN</ul>
<ul>Gated Recurrent Unit (GRU)</ul>
<ul>Long Short-Term Memory (LSTM)</ul>

<h2>2. Hyperparameter Tuning</h2>
<ul>Optimize hyperparameters for each model to achieve the best performance.</ul>


In [6]:
class TextDataset(Dataset):
    def __init__(self, texts, scores):
        self.texts = texts
        self.scores = scores

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        return self.texts[idx], self.scores[idx]

In [7]:
class RNNModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNNModel, self).__init__()
        self.rnn = nn.RNN(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        out, _ = self.rnn(x)
        out = self.fc(out[:, -1, :])
        return out

class BiRNNModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(BiRNNModel, self).__init__()
        self.rnn = nn.RNN(input_size, hidden_size, batch_first=True, bidirectional=True)
        self.fc = nn.Linear(hidden_size * 2, output_size)

    def forward(self, x):
        out, _ = self.rnn(x)
        out = self.fc(out[:, -1, :])
        return out

class GRUModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(GRUModel, self).__init__()
        self.gru = nn.GRU(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        out, _ = self.gru(x)
        out = self.fc(out[:, -1, :])
        return out

class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        out, _ = self.lstm(x)
        out = self.fc(out[:, -1, :])
        return out


In [8]:
def train_model(model, dataloader, criterion, optimizer, num_epochs=10):
    model.train()
    for epoch in range(num_epochs):
        for texts, scores in dataloader:
            texts = texts.float()
            scores = scores.float().view(-1, 1)

            outputs = model(texts)
            loss = criterion(outputs, scores)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

<h1>Part 4: Model Evaluation</h1>

<h2>1. Evaluate Models Using Standard Metrics</h2>
<ul>Precision, Recall, F1-Score, etc.</ul>


<h2>2. Evaluate Models Using BLEU Score</h2>
<ul>Specifically used to evaluate the quality of text which has been machine-translated from one language to another.</ul>

In [9]:
def evaluate_model(model, dataloader):
    model.eval()
    predictions = []
    actuals = []
    with torch.no_grad():
        for texts, scores in dataloader:
            texts = texts.float()
            scores = scores.float().view(-1, 1)

            outputs = model(texts)
            predictions.extend(outputs.numpy())
            actuals.extend(scores.numpy())

    mse = mean_squared_error(actuals, predictions)
    mae = mean_absolute_error(actuals, predictions)
    return mse, mae


In [10]:

preprocessed_texts = [np.random.rand(10, 100) for _ in range(2)] 
scores = [6, 7.5]

train_texts, test_texts, train_scores, test_scores = train_test_split(preprocessed_texts, scores, test_size=0.2, random_state=42)

train_texts = torch.tensor(train_texts, dtype=torch.float32)
test_texts = torch.tensor(test_texts, dtype=torch.float32)
train_scores = torch.tensor(train_scores, dtype=torch.float32)
test_scores = torch.tensor(test_scores, dtype=torch.float32)

train_dataset = TextDataset(train_texts, train_scores)
test_dataset = TextDataset(test_texts, test_scores)
train_dataloader = DataLoader(train_dataset, batch_size=2, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=2, shuffle=False)

input_size = 100  
hidden_size = 128
output_size = 1
num_epochs = 10
learning_rate = 0.001

models = {
    'RNN': RNNModel(input_size, hidden_size, output_size),
    'BiRNN': BiRNNModel(input_size, hidden_size, output_size),
    'GRU': GRUModel(input_size, hidden_size, output_size),
    'LSTM': LSTMModel(input_size, hidden_size, output_size)
}

for model_name, model in models.items():
    print(f'Training {model_name} model...')
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    train_model(model, train_dataloader, criterion, optimizer, num_epochs)

    mse, mae = evaluate_model(model, test_dataloader)
    print(f'{model_name} model Evaluation:')
    print(f' - Mean Squared Error: {mse:.4f}')
    print(f' - Mean Absolute Error: {mae:.4f}')

<ipython-input-10-583816fe1dcb>:9: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:274.)
  train_texts = torch.tensor(train_texts, dtype=torch.float32)


Training RNN model...
Epoch [1/10], Loss: 37.3872
Epoch [2/10], Loss: 30.3672
Epoch [3/10], Loss: 24.2462
Epoch [4/10], Loss: 19.0473
Epoch [5/10], Loss: 14.7069
Epoch [6/10], Loss: 11.1359
Epoch [7/10], Loss: 8.2524
Epoch [8/10], Loss: 5.9749
Epoch [9/10], Loss: 4.2155
Epoch [10/10], Loss: 2.8851
RNN model Evaluation:
 - Mean Squared Error: 11.5938
 - Mean Absolute Error: 3.4050
Training BiRNN model...
Epoch [1/10], Loss: 34.7477
Epoch [2/10], Loss: 27.6209
Epoch [3/10], Loss: 21.3573
Epoch [4/10], Loss: 15.9528
Epoch [5/10], Loss: 11.4255
Epoch [6/10], Loss: 7.7738
Epoch [7/10], Loss: 4.9554
Epoch [8/10], Loss: 2.8894
Epoch [9/10], Loss: 1.4721
Epoch [10/10], Loss: 0.5912
BiRNN model Evaluation:
 - Mean Squared Error: 5.8742
 - Mean Absolute Error: 2.4237
Training GRU model...
Epoch [1/10], Loss: 36.0903
Epoch [2/10], Loss: 31.6113
Epoch [3/10], Loss: 27.4232
Epoch [4/10], Loss: 23.4691
Epoch [5/10], Loss: 19.7292
Epoch [6/10], Loss: 16.2229
Epoch [7/10], Loss: 12.9955
Epoch [8/10], 